## Node2Vec 笔记
>author: xumayi@m.scnu.edu.cn
### 先前的方法存在的问题
没有给节点采样提供足够灵活的策略，全部都是以一种比较固定的策略对目标节点的邻居节点去采样（e.g., DeepWalk, LINE），这种做法对网络中的一些不同部位之间的联系会极其不敏感。
如下图所示，$u$和$s_1$同属于一个极其小的社区范围，它们应该具备类似的表示，$u$和$s_6$距离很远，但$u$和$s_6$在它们所在的临近网络中占据了相同的地位，这里可以
理解为它们具备类似的上下文，因此它们也应该具备类似的表示。

<img src = 'https://i.bmp.ovh/imgs/2021/11/c70fa4f5e6a2156d.jpg' width='500'>

### Feature learning framework
定义网络$G=(V,E)$，我们的目的就是要学习一个函数
$f:V \rightarrow \mathbb{R}^d$，它可以将节点映射到特征表示，$d$是特征
表示的维度。$f$实质上是维度为$|V| \times d$的矩阵，对一个节点$u \in V$，定义$N_{S}(u) \subset V$为采用邻居采样策略$S$对节点$u$进行
网络邻居节点采样的结果。

优化目标：
$\displaystyle\max _{f} \displaystyle\sum_{u \in V} \log \operatorname{Pr}\left(N_{S}(u) \mid f(u)\right) \tag{1} $

即最大化利用$f(u)$预测$N_S(u)$的概率

为了减少计算开销，通常进行如下假设：

* 条件独立：通过源节点来预测每一个领域节点是独立的  $\operatorname{Pr}\left(N_{S}(u) \mid f(u)\right)=\displaystyle \prod_{n_{i} \in N_{S}(u)} \operatorname{Pr}\left(n_{i} \mid f(u)\right)$

* 特征空间对称性：$\operatorname{Pr}\left(n_{i} \mid f(u)\right)=\frac{\exp \left(f\left(n_{i}\right) \cdot f(u)\right)}{\sum_{v \in V} \exp (f(v) \cdot f(u))}$ (这一步可参考Word2Vec中的计算)

利用这两条性质对公式$(1)$进行优化，

$\displaystyle\max _{f} \displaystyle\sum_{u \in V} \log \operatorname{Pr}\left(N_{S}(u) \mid f(u)\right) = \displaystyle\max _{f} \displaystyle\sum_{u \in V} \log (\displaystyle \prod_{n_{i} \in N_{S}(u)}\operatorname{Pr}\left(n_{i} \mid f(u)\right))$

$ = \displaystyle\max _{f} \displaystyle\sum_{u \in V} \log (\displaystyle \prod_{n_{i} \in N_{S}(u)}\operatorname{\frac{\exp \left(f\left(n_{i}\right) \cdot f(u)\right)}{\sum_{v \in V} \exp (f(v) \cdot f(u))}}) = \max _{f} \sum_{u \in V}\left[-\log Z_{u}+\sum_{n_{i} \in N_{S}(u)} f\left(n_{i}\right) \cdot f(u)\right] .$

其中$Z_{u}=\sum_{v \in V} \exp (f(u) \cdot f(v))$，$Z_(u)$直接计算成本过高，所以采用负采样来进行优化。

#### 经典采样策略

首先，为了公平的比较各个采样策略，应该设定相同的采样数量，即每种采样算法每次均选取$k$个节点。采样策略通常可分为两类：

* 广度优先：$N_S$中的节点是目标采样节点的直接临近节点，如图1，如果要对节点$u$进行广度优先采样，当采样窗口$k=3$时，那么$N_S(u)=\{s_1, s_2, s_3\}$
* 深度优先：采样集合$N_S$中的节点离源节点的距离依次增加，在图1中，对节点$u$进行广度优先采样，同样$k=3$的情况下，$N_S(u)=\{s_4, s_5, s_6\}$

通常来讲，在网络上进行预测任务应遵循以下两点性质

**同质性**（homophily）：高度邻近的节点应具备类似的表示

**结构对等性**（structural equivalence）：在相似的结构中具备类似的低微也应该具备类似的表示，例如图1中$u$和$s_6$在它们对应的社区网络中都出于中心地位。

不同于同质性的是，结构对等性并不强调节点之间的连接性，节点之间可以间隔非常远且具备相同的结构地位。

## Method

一个源节点$u$，假定随机游走的长度为$l$，$c_i$为路径上的第$i$个节点，起点$u=c_0$，节点$c_i$由以下分布产生：

$P\left(c_{i}=x \mid c_{i-1}=v\right)=\left\{\begin{array}{ll}\frac{\pi_{v x}}{Z} & \text { if }(v, x) \in E \\ 0 & \text { otherwise }\end{array}\right.$

$\pi_{v x}$是未归一化的由$v$到$x$的概率，$Z$是正则化常量。

* BFS -> homophily
* DFS -> structural equivalence

### Search bias $\alpha$

合适的$\pi_{v x}$对采样至关重要，本文选取的$\pi_{v x}$以$w_{v x}$为基础，即以当前节点和其邻居节点之间的权重为基础。

<img src = 'https://i.bmp.ovh/imgs/2021/11/29ecba3f1289efa8.png' width='500'>

Case: 如图2所示，假定游走路径由$t$游走至$v$且停留在$v$，那么当前停留的节点$v$与其邻居节点之间的概率为：

$\pi_{v x}=\alpha_{p q}(t, x) \cdot w_{v x}$

其中：

$\alpha_{p q}(t, x)=\left\{\begin{array}{ll}\frac{1}{p} & \text { if } d_{t x}=0 \\ 1 & \text { if } d_{t x}=1 \\ \frac{1}{q} & \text { if } d_{t x}=2\end{array}\right.$

$d_{tx}$为上一个路过的节点$t$到当前节点$v$的邻居节点的距离。

* $p$ (Return parameter): 当$p$取较大值时 ($> max(q,1)$) ，此时采样策略将更偏向于对新的节点进行采样，当$p$取较大值时 ($< min(q,1)$)，此时的采样策略将偏向于向刚刚已经采样了的节点进行采样，也就是更偏向于对$t$进行采样。
* $q$ (In-out parameter): 当$q>1$时，此时采样策略将偏向于对与$t$较近的节点进行采样，当$q<1$时，此时采样策略将偏向于对与$t$较远的节点进行采样。


## The Node2Vec algorithm
<img src="https://cdn.mathpix.com/snip/images/H39-mu7V0Oa9WltkXu0mwBuO_G-rh-H4xDqpCYStxOU.original.fullsize.png" width='500'>

为了解决选取起始节点$u$所带来的bias，该算法对每个节点都进行$r$次步长为$l$的采样。所有的采样过程都基于预先计算好的$\pi_{vx}$。

***
**补充知识：AliasSample:**

假定有以下四种4种情况，每种的概率如下所示

<img src="https://cdn.mathpix.com/snip/images/_g9iWNlp9LiuURY4vSRy-45oL9EkQ95GcNui17pJUE8.original.fullsize.png" width='400'>

1）将每种情况的概率 $\times$ 情况的数量，每种情况的概率变为如下图所示，这样总概率就是4

<img src = 'https://i.bmp.ovh/imgs/2021/11/0c6f7f28ab152a2a.png' width='380'>

2）把这四种情况重新拼成二项分布，每一列的概率值为1，且最多只能包含两种情况，如下图所示

<img src = 'https://i.bmp.ovh/imgs/2021/11/6071e10e7115909d.png' width ='380'>

3）先以等概率分布抽取四列中的一列，然后第二步生成一个在区间$[0,1]$的值，在第一步抽样得到的列中继续抽样(这里就变为了二项分布抽样)。

***

### Learning edge features

Node2Vec算法采用一种半监督的方法学习到了网络中的节点表示，但许多任务需要用到节点与节点之间的联系，因此本文构建了不同的$g(u,v)$就对节点$u$与$v$之间的联系进行了表示。$g: V \times V \rightarrow \mathbb{R}^{d^\prime}$，$d^\prime$是节点间表示的维度。
本文选取一些不同的函数$g$ ，如下表所示。

<img src = 'https://i.bmp.ovh/imgs/2021/11/d98413cab526e997.png' width='500'>